<a href="https://colab.research.google.com/github/wissamismail/Manifest_PDF_Extractor/blob/master/MyLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#Basic Libraries
import warnings
import glob #List Directory Files
import pandas as pd # Read CSV File
import numpy as np
#Graphic displays Libraries
import matplotlib.pyplot as plt
from keras.utils import plot_model
#Model Libraries
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from keras.regularizers import l2
from keras import optimizers, Sequential
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from tensorflow.keras.models import save_model, load_model

In [4]:
# list all hwinfo raw csv files only in the specified path
Path = "D:/Log Files/Ready"
csv_files = glob.glob(Path + "/*.csv")#'*.{}'.format('csv'))
csv_files

[]

In [ ]:
df=pd.read_csv(Path+"/Unique-Scaled-38Featurs.csv",encoding='unicode-escape')
df.head(2)

In [ ]:
df.dtypes

In [ ]:
df.shape

In [ ]:
attack = df[df.Label == 1]
normal = df[df.Label == 0]

In [ ]:
data = df.loc[:, df.columns != 'Label'].values  # converts the df to a numpy array
labels = df['Label'].values

n_features = data.shape[1]  # number of features
print(f"NB of Dataset Features: {n_features}")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=1)

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

In [ ]:
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"X_valid shape: {X_valid.shape}")

In [ ]:
timesteps = 1
X_train = X_train.reshape(X_train.shape[0], timesteps, n_features)
X_test = X_test.reshape(X_test.shape[0], timesteps, n_features)
X_valid = X_valid.reshape(X_valid.shape[0], timesteps, n_features)
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"X_valid shape: {X_valid.shape}")

In [ ]:
from tensorflow import keras
LAYERS = [37, 16, 37, 1]                # number of units in hidden and output layers
LSTMAmodel = Sequential()
LSTMAmodel.add(LSTM(input_shape=(timesteps, n_features), units=LAYERS[0], activation='tanh', recurrent_activation='hard_sigmoid',
               kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01),
               dropout=0.2, recurrent_dropout=0.2,
               return_sequences=True, return_state=False,
               stateful=False, unroll=False
              ))

LSTMAmodel.add(BatchNormalization())                   # kernel_regularizer acts on the weights
LSTMAmodel.add(LSTM(units=LAYERS[2],
               activation='tanh', recurrent_activation='hard_sigmoid',
               kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01),
               dropout=0.2, recurrent_dropout=0.2,
               return_sequences=True, return_state=False,
               stateful=False, unroll=False
              ))

LSTMAmodel.add(BatchNormalization())
LSTMAmodel.add(LSTM(units=LAYERS[2],
               activation='tanh', recurrent_activation='hard_sigmoid',
               kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01),
               dropout=0.2, recurrent_dropout=0.2,
               return_sequences=False, return_state=False,
               stateful=False, unroll=False
              ))

LSTMAmodel.add(BatchNormalization())
LSTMAmodel.add(Dense(units=LAYERS[3], activation='sigmoid'))

# Compile the model with Adam optimizer
#LSTMAmodel.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')
LSTMAmodel.compile(loss='binary_crossentropy', optimizer='adamax', metrics=['accuracy'])#,keras.metrics.Precision(),keras.metrics.Recall()])

LSTMAmodel.summary()

In [ ]:
plot_model(LSTMAmodel, show_shapes=True, dpi = 100)

In [ ]:
es =EarlyStopping(patience=3, monitor='val_loss', restore_best_weights=True, mode='min')
LSTMAmodel_history = LSTMAmodel.fit(X_train, y_train, epochs=100,
                                                batch_size=32, validation_data=(X_valid, y_valid)
                                                , callbacks=[es],
                                                shuffle=False).history

In [ ]:
plt.plot(LSTMAmodel_history['loss'], linewidth=2, label='Train')
plt.plot(LSTMAmodel_history['val_loss'], linewidth=2, label='Valid')
plt.legend(loc='upper right')
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.show()

In [ ]:
Path = "D:/Log Files/Ready"
save_model(LSTMAmodel, Path+'/MyLSTM-V01.keras')
print("Saved model to disk")

In [ ]:
# Evaluate the model:
train_loss, train_acc = LSTMAmodel.evaluate(X_train, y_train)
test_loss, test_acc = LSTMAmodel.evaluate(X_test,y_test)
valid_loss, valid_acc = LSTMAmodel.evaluate(X_valid,y_valid)

In [ ]:
print(f'training accuracy = {round(train_acc * 100, 4)}%')
print(f'testing accuracy = {round(test_acc * 100, 4)}%')
print(f'validation accuracy = {round(valid_acc * 100, 4)}%')
print(f'train errors = {round((1 - train_acc) * X_train.shape[0])} out of {X_train.shape[0]} samples')
print(f'test errors = {round((1 - test_acc) * X_test.shape[0])} out of {X_test.shape[0]} samples')


In [ ]:
#predictions = LSTMAmodel.predict(X_valid[:600]) #using 600 rows only because RAM limitation
y_pred= LSTMAmodel.predict(X_valid)

In [ ]:
# Set a threshold for anomaly detection (you might need to adjust this based on your specific case)
threshold = 0.5

In [ ]:
# Apply thresholding to convert autoencoder output to binary labels
y_pred_binary = (y_pred > threshold).astype(int)

In [ ]:
from sklearn.metrics import confusion_matrix
# Compute confusion matrix
conf_matrix = confusion_matrix(y_valid, y_pred_binary)

In [ ]:
# Extract TP, FP, TN, FN from the confusion matrix
TP = conf_matrix[1, 1]
FP = conf_matrix[0, 1]
TN = conf_matrix[0, 0]
FN = conf_matrix[1, 0]

In [ ]:
# Compute precision, recall, and F1-score
precision = precision_score(y_valid, y_pred_binary)
recall = recall_score(y_valid, y_pred_binary)
f1 = f1_score(y_valid, y_pred_binary)

In [ ]:
# Print the results
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-Score: {f1:.4f}')